<a href="https://colab.research.google.com/github/tdelp/AI_Project/blob/main/AI_Project_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Final code

In [ ]:
import random

# Card and their values in Blackjack
deck = {"Ace": 4, "Two": 4, "Three": 4, "Four": 4, "Five": 4, "Six": 4, "Seven": 4, "Eight": 4, "Nine": 4, "Ten": 16}
deckval = {"Ace": 11, "Two": 2, "Three": 3, "Four": 4, "Five": 5, "Six": 6, "Seven": 7, "Eight": 8, "Nine": 9, "Ten": 10}

# Function to calculate percentages of each card in the deck
def calculate_percentages(deck):
    total_cards = sum(deck.values())
    percentages = {card: (count / total_cards) * 100 for card, count in deck.items()}
    return percentages

# Function to make a weighted random selection based on the card percentages
def weighted_random_selection(deck, percentages):
    return random.choices(list(deck.keys()), weights=list(percentages.values()), k=1)[0]

# Function to calculate the odds of busting if another card is taken
def calculate_odds_of_busting(current_total, deck):
    odds = 0
    for card, value in deckval.items():
        if current_total + (1 if card == 'Ace' else value) > 21:
            odds += deck[card]
    total_cards = sum(deck.values())
    return odds / total_cards if total_cards else 0

# Function to draw a card from the deck
def draw_card(deck, current_total, player_type="player"):
    card = weighted_random_selection(deck, calculate_percentages(deck))
    value = 1 if card == 'Ace' and current_total + deckval[card] > 21 else deckval[card]
    current_total += value
    deck[card] -= 1
    print(f"{player_type.capitalize()} drew a {card}.")
    print(f"{player_type.capitalize()}'s new total is {current_total}.\n")
    return current_total

# Function to handle the bot's strategy
def bot_turn(bot_total, bot_name, deck):
    while bot_total < 17:
        odds_of_busting = calculate_odds_of_busting(bot_total, deck)
        print(f"{bot_name.capitalize()} has a {odds_of_busting:.2%} chance of busting.")
        if odds_of_busting < 0.5:
            bot_total = draw_card(deck, bot_total, player_type=bot_name)
            if bot_total > 21:
                print(f"{bot_name.capitalize()} busted!\n")
                break
        else:
            print(f"{bot_name.capitalize()} stands with a total of {bot_total}.\n")
            break
    if bot_total >= 17 and bot_total <= 21:
        print(f"{bot_name.capitalize()} stands with a total of {bot_total}.\n")
    return bot_total

# Function to handle the player's turn
def player_turn(player_total, deck):
    while player_total < 21:
        odds_of_busting = calculate_odds_of_busting(player_total, deck)
        print(f"You have a {odds_of_busting:.2%} chance of busting.")
        choice = input("Would you like to hit or stand? ").lower()
        if choice == "hit":
            player_total = draw_card(deck, player_total)
            if player_total > 21:
                print("You busted!\n")
                break
        elif choice == "stand":
            print(f"You stand with a total of {player_total}.\n")
            break
    return player_total

# Function to handle the dealer's turn
def dealer_turn(dealer_total, deck):
    dealer_busted = False
    while dealer_total < 17:
        dealer_total = draw_card(deck, dealer_total, player_type="dealer")
        if dealer_total > 21:
            print("Dealer busted!\n")
            dealer_busted = True
            break
    return dealer_total, dealer_busted

# Main function to run the Blackjack game
def play_blackjack(num_bots):
    player_total = 0
    dealer_total = 0
    bot_totals = [0] * num_bots

    # Deal initial cards in rounds
    for _ in range(2):  # Two rounds of dealing
        # Deal one card to player
        player_total = draw_card(deck, player_total, "player")

        # Deal one card to each bot
        for i in range(num_bots):
            bot_totals[i] = draw_card(deck, bot_totals[i], f"bot {i+1}")

        # Deal one card to dealer
        dealer_total = draw_card(deck, dealer_total, "dealer")

    # Player's turn
    print("Your turn:")
    player_total = player_turn(player_total, deck)

    # Bots' turns
    for i in range(num_bots):
        print(f"Bot {i+1}'s turn:")
        bot_totals[i] = bot_turn(bot_totals[i], f"bot {i+1}", deck)

    # Dealer's turn
    print("Dealer's turn:")
    dealer_total, dealer_busted = dealer_turn(dealer_total, deck)

    # Results and comparison
    print("Final results:\n")

    # Display the dealer's outcome
    dealer_outcome = 'busted' if dealer_busted else 'stands'
    print(f"Dealer {dealer_outcome} with a total of {dealer_total}\n")

    # Display the player's outcome
    player_outcome = 'busted' if player_total > 21 else 'stands'
    print(f"You {player_outcome} with a total of {player_total}\n")

    # Display the bots' outcomes
    for i in range(num_bots):
        bot_outcome = 'busted' if bot_totals[i] > 21 else 'stands'
        print(f"Bot {i+1} {bot_outcome} with a total of {bot_totals[i]}\n")

    print("Outcome:\n")

    # Compare the player's total with the dealer's, if neither has busted
    if player_total <= 21 and not dealer_busted:
        if player_total > dealer_total:
            print("You beat the dealer!")
        elif player_total < dealer_total:
            print("You lost to the dealer.")
        else:
            print("It's a tie with the dealer.")
    elif player_total <= 21 and dealer_busted:
        print("You win! Dealer busted.")
    else:
        print("You busted and lost.")

    # Compare each bot's total with the dealer's, if neither has busted
    for i in range(num_bots):
        if bot_totals[i] <= 21 and not dealer_busted:
            if bot_totals[i] > dealer_total:
                print(f"Bot {i+1} beats the dealer!")
            elif bot_totals[i] < dealer_total:
                print(f"Bot {i+1} loses to the dealer.")
            else:
                print(f"Bot {i+1} ties with the dealer.")
        elif bot_totals[i] <= 21 and dealer_busted:
            print(f"Bot {i+1} wins! Dealer busted.")
        else:
            print(f"Bot {i+1} busted and lost.")

# Start the game
num_bots = int(input("Enter the number of bots to play with (0-3): "))
num_bots = max(0, min(num_bots, 3))  # Ensure the number of bots is between 0 and 3
play_blackjack(num_bots)